# 簡單教學 - 建立一個sqlite檔案，整理抓到的csv檔案
這個簡單的教學分為三個部份，第一個部份我們會利用glob以及pandas套件來讀取我們抓來的2014-2019年的台股資料，第二個部份為建立一個sqlite檔案來存入我們的資料庫，第三個部份為進一步整理為用個股來分的資料庫。

In [2]:
import pandas as pd
import sqlite3
import glob

## 第一部份 讀取我們抓到的資料
當我們用EP1的程式抓取大量的表單之後，如果我們簡單用迴圈加to_csv方法，會發現現在我們的資料夾裡面已經堆滿的大量的csv檔案了，我們要如何使用程式來整理我們的檔案勒？如果用pandas一個一個讀入顯然很沒有效率，這邊我們使用glob套件一次讀入所有的csv檔案吧

In [2]:
'''
glob套件是用來查找符合特定規則的文件名，跟我們用搜尋跳出來的結果差不多，這邊我們查詢副檔名為csv的檔案並存為一個列表的形式。
'''
All_csv_file = glob.glob('*.csv')

In [3]:
df = pd.read_csv(All_csv_file[0]).iloc[:,1:]

## 第二部份 創建資料庫，存成以時間為一張張表的資料庫
我們這邊會使用sqlite來存取我們抓下來的股價資料
- python內建sqlite套件，我們無須特安裝
- 支援完整sql語法查詢我們的資料
- 使用以及轉移方便，一個資料庫就像一個本地文件一樣

在這邊，我們直接使用DataFrame提供把DataFrame存入Sql資料庫當作表格的方法。

In [4]:
dbname = 'TWStock.db'

In [5]:
#連接到我們的資料庫，如果沒有的話會重新建一個
db = sqlite3.connect(dbname)

In [6]:
%%time
for file_name in All_csv_file:
    pd.read_csv(file_name).iloc[:,1:].to_sql(file_name.replace('.csv',''),db,if_exists='replace')

CPU times: user 47.7 s, sys: 3.39 s, total: 51.1 s
Wall time: 1min 11s


 ### 如何讀取資料庫的表格
我們這邊簡單介紹如何讀取sqlite檔案裡面的表格

In [7]:
dates_list = [file_name.replace('.csv','') for file_name in All_csv_file]

In [8]:
pd.read_sql(con=db,sql='SELECT * FROM' + ' "'+ dates_list[0] +'"').head()

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0,0050,元大台灣50,"8,909,455","1,051","595,197,353",66.75,66.90,66.65,66.85,,0.00,66.85,385,66.90,136,0.00
1,1,0051,元大中型100,"26,120",21,"754,990",28.89,28.94,28.83,28.88,+,0.01,28.88,1,28.90,1,0.00
2,2,0052,富邦科技,"2,000",2,"81,960",40.98,40.98,40.98,40.98,+,0.14,40.33,1,40.98,3,0.00
3,3,0053,元大電子,"45,466",20,"1,294,287",28.59,28.59,28.45,28.50,,0.00,28.45,8,28.54,2,0.00
4,4,0054,元大台商50,"24,000",5,"537,140",22.37,22.44,22.35,22.44,-,0.01,22.22,3,22.44,1,0.00


## 第三部份 整理出以個股為一張張表的資料庫

In [15]:
%%time
total_df = pd.DataFrame()
for date in dates_list:
    df = pd.read_sql(con=db,sql='SELECT * FROM' + ' "'+ date +'"')
    df['Date'] = date
    total_df = total_df.append(df)

CPU times: user 19min 19s, sys: 2min 53s, total: 22min 13s
Wall time: 22min 13s


In [16]:
%%time
total_df.shape

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 17.6 µs


(1203195, 18)

In [3]:
dbname_2 = 'TWStock_2'
db2 = sqlite3.connect(dbname_2)

In [48]:
%%time 
total_dict = dict(tuple(total_df.groupby('證券代號')))
for key in total_dict.keys():
    df = total_dict[key].iloc[:,2:]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Date'])
    df.to_sql(key,db2,if_exists='replace')

In [7]:
%%time
#測試股票代號資料庫
print(pd.read_sql(con=db2,sql='SELECT * FROM "2330"').tail())
print(pd.read_sql(con=db2,sql='SELECT * FROM "6422"').tail())

      index 證券名稱        成交股數    成交筆數           成交金額     開盤價     最高價     最低價  \
1216    410  台積電  24,443,428  11,779  5,745,045,809  237.50  238.00  233.00   
1217    410  台積電  35,901,584  12,064  8,461,930,934  238.00  238.00  234.00   
1218    410  台積電  34,651,731  14,723  8,008,768,323  231.50  232.00  229.50   
1219    410  台積電  35,521,888  15,588  8,442,986,620  237.50  240.00  234.50   
1220    410  台積電  34,691,670  13,157  8,448,805,962  240.00  246.00  239.00   

         收盤價 漲跌(+/-)  漲跌價差  最後揭示買價 最後揭示買量  最後揭示賣價 最後揭示賣量    本益比  \
1216  233.00       -   5.0  233.00  3,359  233.50    136  18.71   
1217  235.00       +   2.0  234.50     81  235.00    661  18.88   
1218  232.00       -   3.0  232.00     91  232.50    824  18.63   
1219  240.00       +   8.0  239.50    207  240.00    817  19.28   
1220  244.50       +   4.5  244.00  1,076  244.50     55  19.64   

                     Date  
1216  2019-06-04 00:00:00  
1217  2019-06-05 00:00:00  
1218  2019-06-06 00:00:00  
1219  2019